In [1]:
import pandas as pd
import numpy as np
import os

### Join Sentinel NDVI or EVI data to main dataframe
##### Prior to reading in ndvi dataframe, create date column to match each measurement

In [2]:
data_path = r'C:\Users\roseh\Desktop\NYBG_R\data'

In [29]:
df = pd.read_csv(os.path.join(data_path,'nybg_all_main.csv'),parse_dates=['date'])

In [11]:
evi_df = pd.read_csv(os.path.join(data_path,'evi_may_nov_nybg.csv'),parse_dates=['res_date'])

In [12]:
evi_df = evi_df.loc[evi_df['res_date'].notnull()]
evi_df = pd.melt(evi_df,id_vars='res_date',value_vars=['BE_UL1','BL_ML1','BR_HL3','BT_FE2','BT_FE3','BT_FI1','BT_FI3','BW_HL1','BE_UL3','BG_ML3'])
evi_df.rename(columns={'value':'evi'},inplace=True)
evi_df_grouped = evi_df.groupby('res_date')

In [14]:
ndvi_df = pd.read_csv(os.path.join(data_path,'ndvi_sentinel_nybg_may_nov.csv'),parse_dates=['res_date'])

In [15]:
ndvi_df = ndvi_df.loc[ndvi_df['res_date'].notnull()]
ndvi_df = pd.melt(ndvi_df,id_vars='res_date',value_vars=['BE_UL1','BL_ML1','BR_HL3','BT_FE2','BT_FE3','BT_FI1','BT_FI3','BW_HL1','BE_UL3','BG_ML3'])
ndvi_df.rename(columns={'value':'ndvi'},inplace=True)
ndvi_df_grouped = ndvi_df.groupby('res_date')

In [25]:
ndvi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   res_date  140 non-null    datetime64[ns]
 1   variable  140 non-null    object        
 2   ndvi      140 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 3.4+ KB


In [32]:
df_grouped = df.groupby('date')

In [33]:
df_list = []
for name,group in df_grouped:
    group_df = pd.DataFrame(group)
    merge_df = group_df.merge(pd.DataFrame(evi_df_grouped.get_group(name)),
                              left_on='Label',right_on='variable',how='outer').merge(pd.DataFrame(ndvi_df_grouped.get_group(name)),
                                                                                     left_on='Label',right_on='variable',how='outer')
    df_list.append(merge_df)       

In [34]:
combine=pd.concat(df_list)

In [35]:
combine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395 entries, 0 to 28
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Unnamed: 0.1      337 non-null    float64       
 1   Unnamed: 0        337 non-null    float64       
 2   date              363 non-null    datetime64[ns]
 3   Label             363 non-null    object        
 4   Res               235 non-null    float64       
 5   NEE               71 non-null     float64       
 6   GEE               77 non-null     float64       
 7   PAR               64 non-null     float64       
 8   Soil temperature  218 non-null    float64       
 9   Soil_moist        204 non-null    float64       
 10  type              363 non-null    object        
 11  Chamber_Temp      224 non-null    float64       
 12  sentinel_ndvi     121 non-null    float64       
 13  hh_ndvi           28 non-null     float64       
 14  sentinel_evi      121 non-n

In [37]:
combine['sentinel_evi'] = combine['evi']
combine['sentinel_ndvi'] = combine['ndvi']

In [38]:
combine = combine.drop(['res_date_x','variable_x','res_date_y','variable_y'],axis=1)

In [42]:
combine = combine.drop(['evi','ndvi'],axis=1)

In [53]:
combine = combine.loc[combine['date'].notnull()]

In [56]:
combine.to_csv(os.path.join(data_path,'nybg_all_main.csv'))